In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

pwd = os.getcwd()

In [2]:
ssd_results_path = str("/work1/fwk/sajid/hepnos_es/scaling/ssd_hepnos/servers_one_eigth/multi_servers_per_node")

In [3]:
nodes = np.array([8,16,32,64,128,256])

In [4]:
times_ssd = pd.DataFrame(
    {
        "files" : [],
        "server_nodes_fraction" : [],
        "server_rpc_xstreams_per_MPI_rank" : [],
        "total_num_nodes" : [],
        "MPI_rank_among_clients" : [],
        "expt_number" : [],
        "analysis_start_times" : [],
        "pep_start_times" : [],
        "pep_end_times" : [],
        "analysis_end_times" : [],
        "analysis_method" : [],
    }
)

In [5]:
#1929-files

#8,16,32,64,128,256
es1929_pep_start_times = [[],[],[],[],[],[]]
es1929_pep_end_times = [[],[],[],[],[],[]]
es1929_analysis_start_times = [[],[],[],[],[],[]]
es1929_analysis_end_times = [[],[],[],[],[],[]]
es1929_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results are at:
# /work1/fwk/sajid/hepnos_es/ssd_es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/multi_servers_per_node
results_8_nodes = str("/work1/fwk/sajid/hepnos_es/ssd_es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/multi_servers_per_node/es_1929/es_1929_10_13_22")
with os.scandir(results_8_nodes) as it:
    _pep_start_times = []
    _pep_end_times = []
    _analysis_start_times = []
    _analysis_end_times = []
    _mpi_rank_among_clients = []
    for entry in it:
        if entry.name.startswith('timing') and entry.is_file():
            f = open(entry)
            l = f.readlines()
            for k in range(len(l)):
                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                    _start_time = float(l[k].split(",")[0])
                    _end_time = float(l[k+1].split(",")[0])
                    _pep_start_times.append(_start_time)
                    _pep_end_times.append(_end_time)
                    _analysis_start_times.append(float(l[0].split(",")[0]))
                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
    _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
    # Skip failed runs
    if(_time_avg > 0):
        print("run with {} nodes, avg time is {}".format(8, _time_avg))
        es1929_pep_start_times[0].append(_pep_start_times)
        es1929_pep_end_times[0].append(_pep_end_times)
        es1929_analysis_start_times[0].append(_analysis_start_times)
        es1929_analysis_end_times[0].append(_analysis_end_times)
        es1929_mpi_rank_among_clients[0].append(_mpi_rank_among_clients)
        
_rows = len(np.array(es1929_pep_start_times[0]).flatten())
_df_8_ranks = pd.DataFrame(
    {
        "pep_start_times": np.array(es1929_pep_start_times[0]).flatten(),
        "pep_end_times" : np.array(es1929_pep_end_times[0]).flatten(),
        "analysis_start_times": np.array(es1929_analysis_start_times[0]).flatten(),
        "analysis_end_times": np.array(es1929_analysis_end_times[0]).flatten(),
        "expt_number" : np.ones(_rows, dtype=np.int32)*1,
        "MPI_rank_among_clients" : np.array(es1929_mpi_rank_among_clients[0]).flatten(),
        "files" : np.ones(_rows, dtype=np.int32)*1929,
        "total_num_nodes" : np.ones(_rows, dtype=np.int32)*8,
        "server_nodes_fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
        "server_rpc_xstreams_per_MPI_rank" : np.ones(_rows, dtype=np.int32)*16,
        "analysis_method" : str("HEPnOS-rocksdb"),
    }
)
times_ssd = pd.concat([times_ssd, _df_8_ranks], ignore_index=True)


#16,32,64,128,256
with os.scandir(ssd_results_path + str("/es_1929/")) as it:
    dirs = []
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(ssd_results_path+str("/es_1929/"))
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            _path = os.path.join(os.getcwd()+"/es_1929_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_start_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_start_times.append(float(l[0].split(",")[0]))
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run with {} nodes, avg time is {}".format(_nodes, _time_avg))
                    es1929_pep_start_times[i+1].append(_pep_start_times)
                    es1929_pep_end_times[i+1].append(_pep_end_times)
                    es1929_analysis_start_times[i+1].append(_analysis_start_times)
                    es1929_analysis_end_times[i+1].append(_analysis_end_times)
                    es1929_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
                    
                    _rows = len(np.array(es1929_pep_start_times[i+1][-1]).flatten())
                    _df = pd.DataFrame(
                        {
                            "pep_start_times": np.array(es1929_pep_start_times[i+1][-1]).flatten(),
                            "pep_end_times" : np.array(es1929_pep_end_times[i+1][-1]).flatten(),
                            "analysis_start_times": np.array(es1929_analysis_start_times[i+1][-1]).flatten(),
                            "analysis_end_times": np.array(es1929_analysis_end_times[i+1][-1]).flatten(),
                            "MPI_rank_among_clients" : np.array(es1929_mpi_rank_among_clients[i+1][-1]).flatten(),
                            "files" : np.ones(_rows, dtype=np.int32)*1929,
                            "expt_number" : np.ones(_rows, dtype=np.int32)*j,
                            "total_num_nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                            "server_nodes_fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                            "server_rpc_xstreams_per_MPI_rank" : np.ones(_rows, dtype=np.int32)*16,
                            "analysis_method" : str("HEPnOS-rocksdb"),
                        }
                    )
                    times_ssd = pd.concat([times_ssd, _df], ignore_index=True)

run with 8 nodes, avg time is 204.60499589285715
run with 16 nodes, avg time is 105.2913439732143
run with 16 nodes, avg time is 105.22377959821428
run with 16 nodes, avg time is 104.4102815625
run with 16 nodes, avg time is 105.23966558035714
run with 16 nodes, avg time is 105.72437508928572
run with 32 nodes, avg time is 57.270501540178564
run with 32 nodes, avg time is 56.13715214285714
run with 32 nodes, avg time is 57.34328044642858
run with 32 nodes, avg time is 56.300257611607144
run with 32 nodes, avg time is 58.103002366071415
run with 64 nodes, avg time is 39.274907120535715
run with 64 nodes, avg time is 39.250935625000004
run with 64 nodes, avg time is 41.812956171875
run with 64 nodes, avg time is 41.849663906249994
run with 64 nodes, avg time is 39.13543431919643
run with 128 nodes, avg time is 33.95233011160714
run with 128 nodes, avg time is 34.43972904017857
run with 128 nodes, avg time is 34.256237265624996
run with 128 nodes, avg time is 35.253129453125
run with 128 

In [6]:
#3858-files

#8,16,32,64,128,256
es3858_pep_start_times = [[],[],[],[],[],[]]
es3858_pep_end_times = [[],[],[],[],[],[]]
es3858_analysis_start_times = [[],[],[],[],[],[]]
es3858_analysis_end_times = [[],[],[],[],[],[]]
es3858_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results are at:
# /work1/fwk/sajid/hepnos_es/ssd_es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/multi_servers_per_node
results_8_nodes = str("/work1/fwk/sajid/hepnos_es/ssd_es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/multi_servers_per_node/es_3858/es_3858_10_13_22")
with os.scandir(results_8_nodes) as it:
    _pep_start_times = []
    _pep_end_times = []
    _analysis_start_times = []
    _analysis_end_times = []
    _mpi_rank_among_clients = []
    for entry in it:
        if entry.name.startswith('timing') and entry.is_file():
            f = open(entry)
            l = f.readlines()
            for k in range(len(l)):
                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                    _start_time = float(l[k].split(",")[0])
                    _end_time = float(l[k+1].split(",")[0])
                    _pep_start_times.append(_start_time)
                    _pep_end_times.append(_end_time)
                    _analysis_start_times.append(float(l[0].split(",")[0]))
                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
    _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
    # Skip failed runs
    if(_time_avg > 0):
        print("run with {} nodes, avg time is {}".format(8, _time_avg))
        es3858_pep_start_times[0].append(_pep_start_times)
        es3858_pep_end_times[0].append(_pep_end_times)
        es3858_analysis_start_times[0].append(_analysis_start_times)
        es3858_analysis_end_times[0].append(_analysis_end_times)
        es3858_mpi_rank_among_clients[0].append(_mpi_rank_among_clients)
        
_rows = len(np.array(es3858_pep_start_times[0]).flatten())
_df_8_ranks = pd.DataFrame(
    {
        "pep_start_times": np.array(es3858_pep_start_times[0]).flatten(),
        "pep_end_times" : np.array(es3858_pep_end_times[0]).flatten(),
        "analysis_start_times": np.array(es3858_analysis_start_times[0]).flatten(),
        "analysis_end_times": np.array(es3858_analysis_end_times[0]).flatten(),
        "expt_number" : np.ones(_rows, dtype=np.int32)*1,
        "MPI_rank_among_clients" : np.array(es3858_mpi_rank_among_clients[0]).flatten(),
        "files" : np.ones(_rows, dtype=np.int32)*3858,
        "total_num_nodes" : np.ones(_rows, dtype=np.int32)*8,
        "server_nodes_fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
        "server_rpc_xstreams_per_MPI_rank" : np.ones(_rows, dtype=np.int32)*16,
        "analysis_method" : str("HEPnOS-rocksdb"),
    }
)
times_ssd = pd.concat([times_ssd, _df_8_ranks], ignore_index=True)


#16,32,64,128,256
with os.scandir(ssd_results_path + str("/es_3858/")) as it:
    dirs = []
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(ssd_results_path+str("/es_3858/"))
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            _path = os.path.join(os.getcwd()+"/es_3858_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_start_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_start_times.append(float(l[0].split(",")[0]))
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run with {} nodes, avg time is {}".format(_nodes, _time_avg))
                    es3858_pep_start_times[i+1].append(_pep_start_times)
                    es3858_pep_end_times[i+1].append(_pep_end_times)
                    es3858_analysis_start_times[i+1].append(_analysis_start_times)
                    es3858_analysis_end_times[i+1].append(_analysis_end_times)
                    es3858_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
                    
                    _rows = len(np.array(es3858_pep_start_times[i+1][-1]).flatten())
                    _df = pd.DataFrame(
                        {
                            "pep_start_times": np.array(es3858_pep_start_times[i+1][-1]).flatten(),
                            "pep_end_times" : np.array(es3858_pep_end_times[i+1][-1]).flatten(),
                            "analysis_start_times": np.array(es3858_analysis_start_times[i+1][-1]).flatten(),
                            "analysis_end_times": np.array(es3858_analysis_end_times[i+1][-1]).flatten(),
                            "MPI_rank_among_clients" : np.array(es3858_mpi_rank_among_clients[i+1][-1]).flatten(),
                            "files" : np.ones(_rows, dtype=np.int32)*3858,
                            "expt_number" : np.ones(_rows, dtype=np.int32)*j,
                            "total_num_nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                            "server_nodes_fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                            "server_rpc_xstreams_per_MPI_rank" : np.ones(_rows, dtype=np.int32)*16,
                            "analysis_method" : str("HEPnOS-rocksdb"),
                        }
                    )
                    times_ssd = pd.concat([times_ssd, _df], ignore_index=True)

run with 8 nodes, avg time is 410.3451899107143
run with 16 nodes, avg time is 206.86801156250002
run with 16 nodes, avg time is 207.75381763392858
run with 16 nodes, avg time is 207.5479014732143
run with 32 nodes, avg time is 108.61146129464285
run with 32 nodes, avg time is 108.95168502232143


/work1/fwk/sajid/miniforge3/envs/py3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/work1/fwk/sajid/miniforge3/envs/py3/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


run with 32 nodes, avg time is 110.04302674107144
run with 64 nodes, avg time is 72.81627815848215
run with 64 nodes, avg time is 75.790628203125
run with 64 nodes, avg time is 72.67312138392857
run with 128 nodes, avg time is 57.00744699776785
run with 128 nodes, avg time is 57.401986071428574
run with 256 nodes, avg time is 69.27442603515625
run with 256 nodes, avg time is 69.29673320870536


In [7]:
#7716-files

#8,16,32,64,128,256
es7716_pep_start_times = [[],[],[],[],[],[]]
es7716_pep_end_times = [[],[],[],[],[],[]]
es7716_analysis_start_times = [[],[],[],[],[],[]]
es7716_analysis_end_times = [[],[],[],[],[],[]]
es7716_mpi_rank_among_clients = [[],[],[],[],[],[]]

# 8 node run results are at:
# /work1/fwk/sajid/hepnos_es/ssd_es_output/theta/8-nodes/hepnos-0.6.11/daemon_1_client_7/multi_servers_per_node
# Cannot fit 7716 file dataset on server with 1 node

#16,32,64,128,256
with os.scandir(ssd_results_path + str("/es_7716/")) as it:
    dirs = []
    run_number_offset = 0
    for _dir in it:
        if not _dir.name.startswith('es') and _dir.is_dir():
            dirs.append(_dir)
    if len(dirs)==0:
        dirs.append(ssd_results_path+str("/es_7716/"))
    for _dir in dirs:
        for i in range(5):
            os.chdir(_dir)
            _nodes = nodes[i+1]
            _path = os.path.join(os.getcwd()+"/es_7716_"+str(_nodes))
            for j in np.arange(1,6):
                os.chdir(_path+"/RUN_"+str(j))
                _pep_start_times = []
                _pep_end_times = []
                _analysis_start_times = []
                _analysis_end_times = []
                _mpi_rank_among_clients = []
                with os.scandir(os.getcwd()) as it:
                    for entry in it:
                        if entry.name.startswith('timing') and entry.is_file():
                            f = open(entry)
                            l = f.readlines()
                            for k in range(len(l)):
                                if l[k].split(",")[-1]==str("pre_pep_process\n"):
                                    _start_time = float(l[k].split(",")[0])
                                    _end_time = float(l[k+1].split(",")[0])
                                    _pep_start_times.append(_start_time)
                                    _pep_end_times.append(_end_time)
                                    _analysis_start_times.append(float(l[0].split(",")[0]))
                                    _analysis_end_times.append(float(l[-1].split(",")[0]))
                                    _mpi_rank_among_clients.append(int(entry.name.split('_')[1]))
                _time_avg = np.mean(np.array(_pep_end_times) - np.array(_pep_start_times))
                # Skip failed runs
                if(_time_avg > 0):
                    print("run number {} with {} nodes, avg time is {}".format((j+run_number_offset), _nodes, _time_avg))
                    es7716_pep_start_times[i+1].append(_pep_start_times)
                    es7716_pep_end_times[i+1].append(_pep_end_times)
                    es7716_analysis_start_times[i+1].append(_analysis_start_times)
                    es7716_analysis_end_times[i+1].append(_analysis_end_times)
                    es7716_mpi_rank_among_clients[i+1].append(_mpi_rank_among_clients)
                    
                    _rows = len(np.array(es7716_pep_start_times[i+1][-1]).flatten())
                    _df = pd.DataFrame(
                        {
                            "pep_start_times": np.array(es7716_pep_start_times[i+1][-1]).flatten(),
                            "pep_end_times" : np.array(es7716_pep_end_times[i+1][-1]).flatten(),
                            "analysis_start_times": np.array(es7716_analysis_start_times[i+1][-1]).flatten(),
                            "analysis_end_times": np.array(es7716_analysis_end_times[i+1][-1]).flatten(),
                            "MPI_rank_among_clients" : np.array(es7716_mpi_rank_among_clients[i+1][-1]).flatten(),
                            "files" : np.ones(_rows, dtype=np.int32)*7716,
                            "expt_number" : np.ones(_rows, dtype=np.int32)*(j+run_number_offset),
                            "total_num_nodes" : np.ones(_rows, dtype=np.int32)*_nodes,
                            "server_nodes_fraction" : np.ones(_rows, dtype=np.float32)*(1/8),
                            "server_rpc_xstreams_per_MPI_rank" : np.ones(_rows, dtype=np.int32)*16,
                            "analysis_method" : str("HEPnOS-rocksdb"),
                        }
                    )
                    times_ssd = pd.concat([times_ssd, _df], ignore_index=True)
        run_number_offset = run_number_offset + 1

run number 1 with 16 nodes, avg time is 415.74085727678573
run number 2 with 16 nodes, avg time is 414.25379986607146
run number 1 with 32 nodes, avg time is 213.20653906249998
run number 1 with 64 nodes, avg time is 135.28266299107142
run number 1 with 128 nodes, avg time is 100.44222457589285
run number 1 with 256 nodes, avg time is 94.2790848939732
run number 2 with 16 nodes, avg time is 412.98941241071435
run number 2 with 32 nodes, avg time is 213.57813939732142
run number 2 with 64 nodes, avg time is 135.17595167410715
run number 2 with 128 nodes, avg time is 98.33494973772324
run number 2 with 256 nodes, avg time is 93.12705304129464
run number 3 with 16 nodes, avg time is 414.3608683035714
run number 4 with 16 nodes, avg time is 414.8475662946429
run number 3 with 32 nodes, avg time is 212.99224984375002
run number 3 with 64 nodes, avg time is 137.67823417410713
run number 3 with 128 nodes, avg time is 103.14481955357142
run number 3 with 256 nodes, avg time is 91.9182545256696

In [8]:
times_ssd

,files,server_nodes_fraction,server_rpc_xstreams_per_MPI_rank,total_num_nodes,MPI_rank_among_clients,expt_number,analysis_start_times,pep_start_times,pep_end_times,analysis_end_times,analysis_method
0,1929.0,0.125,16.0,8.0,3.0,1.0,0.000440,1.53910,206.0380,208.938,HEPnOS-rocksdb
1,1929.0,0.125,16.0,8.0,48.0,1.0,0.000440,1.54418,206.2160,208.943,HEPnOS-rocksdb
2,1929.0,0.125,16.0,8.0,17.0,1.0,0.000442,1.54398,206.0600,208.941,HEPnOS-rocksdb
3,1929.0,0.125,16.0,8.0,94.0,1.0,0.000469,1.54146,206.2050,208.950,HEPnOS-rocksdb
4,1929.0,0.125,16.0,8.0,64.0,1.0,0.000476,1.54561,206.2210,208.815,HEPnOS-rocksdb
...,...,...,...,...,...,...,...,...,...,...,...
75035,7716.0,0.125,16.0,128.0,1128.0,4.0,0.000462,2.42509,104.5610,110.059,HEPnOS-rocksdb
75036,7716.0,0.125,16.0,128.0,344.0,4.0,0.000443,2.44636,101.0300,110.040,HEPnOS-rocksdb
75037,7716.0,0.125,16.0,128.0,1046.0,4.0,0.000461,2.44121,104.3970,110.039,HEPnOS-rocksdb
75038,7716.0,0.125,16.0,128.0,827.0,4.0,0.000457,2.44397,102.3550,110.035,HEPnOS-rocksdb


In [9]:
os.chdir(pwd)
times_ssd.to_csv("ssd-hepnos.csv")